In [1]:
# EuRoC data handler
import pandas as pd
import numpy as np
import tensorflow as tf
# from keras import backend as K
import time
import sys

# class euroc_data():
# 	def __init__(self,groundtruth_path):
# 		gt_frame = pd.read_csv(groundtruth_path,'r')
GROUNDTRUTH_PATH = '/home/cdeng/EuRoC/mav0/\
state_groundtruth_estimate0/data.csv'
IMU_PATH = '/home/cdeng/EuRoC/mav0/imu0/data.csv'

In [2]:
gt_dataframe = pd.read_csv(GROUNDTRUTH_PATH, index_col=0)
imu_dataframe = pd.read_csv(IMU_PATH, index_col=0)
# print(gt_dataframe.head(3))
# print('-----------------')
# print(imu_dataframe.head(3))

# Find the start and end time stamp of the ground truth, 
# and extract relevant imu data frame, gt and imu should be same length  
groudtruth_data = gt_dataframe.iloc[:-1,0:7]
start_time, end_time = groudtruth_data.index[0], groudtruth_data.index[-1]

# Use groundtruth_data.iloc[0,:] as reference cordinate frame
groudtruth_data_rela_pos = groudtruth_data.iloc[:,:]-groudtruth_data.iloc[0,:]
print (start_time, end_time)
print(groudtruth_data_rela_pos.shape)
groudtruth_data.head(5)

1403636580838555648 1403636762738555392
(36381, 7)


,p_RS_R_x [m],p_RS_R_y [m],p_RS_R_z [m],q_RS_w [],q_RS_x [],q_RS_y [],q_RS_z []
#timestamp,,,,,,,
1403636580838555648,4.688319,-1.786938,0.783338,0.534108,-0.153029,-0.827383,-0.082152
1403636580843555328,4.688177,-1.786770,0.787350,0.534640,-0.152990,-0.826976,-0.082863
1403636580848555520,4.688028,-1.786598,0.791382,0.535178,-0.152945,-0.826562,-0.083605
1403636580853555456,4.687878,-1.786421,0.795429,0.535715,-0.152884,-0.826146,-0.084391
1403636580858555648,4.687727,-1.786240,0.799484,0.536244,-0.152821,-0.825731,-0.085213


In [3]:
# ***** Handling imu data ***************
# Set threshold to 2 000 000 ns, because the time interval between imu data is 5 000 000 ns
threshold = 1000000
[imu_start_idx, imu_end_idx] = [idx for (idx, timestamp) in enumerate(imu_dataframe.index)  \
    if (abs(timestamp-start_time)<threshold or abs(timestamp-end_time)<threshold)]
print (imu_start_idx, imu_end_idx)
# ---------imu data at t-1 is used to predicate gt at t
# Groundtruth is for image 1-n, [first_image_middle_exp_time, last_image_middle_exp_time], that't why the number of 
# datapoints end with 1.
imu_data = imu_dataframe[imu_start_idx:imu_end_idx].copy()

if (groudtruth_data_rela_pos.shape[0]-1 != imu_data.shape[0]):
    print("'Error: groundtruth and imu data don't have the same lenght'")
print (imu_data.shape)
print (groudtruth_data_rela_pos.shape)

groudtruth_data_rela_pos.iloc[:32,:]
# imu_data.iloc[1:3]
# imu_dataframe[0:6]

216 36596
(36380, 6)
(36381, 7)


,p_RS_R_x [m],p_RS_R_y [m],p_RS_R_z [m],q_RS_w [],q_RS_x [],q_RS_y [],q_RS_z []
#timestamp,,,,,,,
1403636580838555648,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1403636580843555328,-0.000142,0.000168,0.004012,0.000532,0.000039,0.000407,-0.000711
1403636580848555520,-0.000291,0.000340,0.008044,0.001070,0.000084,0.000821,-0.001453
1403636580853555456,-0.000441,0.000517,0.012091,0.001607,0.000145,0.001237,-0.002239
1403636580858555648,-0.000592,0.000698,0.016146,0.002136,0.000208,0.001652,-0.003061
1403636580863555328,-0.000740,0.000879,0.020202,0.002660,0.000261,0.002069,-0.003897
1403636580868555520,-0.000884,0.001057,0.024256,0.003181,0.000304,0.002487,-0.004738
1403636580873555456,-0.001024,0.001229,0.028304,0.003696,0.000349,0.002902,-0.005573
1403636580878555648,-0.001161,0.001394,0.032344,0.004209,0.000402,0.003316,-0.006401


In [4]:
imu_data.head(4)

,w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]
#timestamp [ns],,,,,,
1403636580838555392,-0.206647,0.325329,-0.040492,10.035472,-0.179789,-4.135137
1403636580843555584,-0.215025,0.333009,-0.046775,9.749445,-0.024517,-4.077932
1403636580848555520,-0.224798,0.336499,-0.057945,9.496106,0.024517,-4.077932
1403636580853555456,-0.237365,0.338594,-0.074700,9.210079,0.032689,-4.061588


In [5]:
# After studying the dataset, ignore the last data from ground truth. Because the second last element 
# has the timestamp that happens to be the middle point of image 3661 and 3662 (opened in Office, column index)
datasize = imu_data.shape[0] 

# IMU 200Hz,   1 min = 12000 samples
series_length = 400 # This is 2 seconds
total_num_sequence = datasize//series_length
total_used_size = total_num_sequence*series_length

X = np.array(imu_data[:total_used_size])
y = np.empty(groudtruth_data_rela_pos[:total_used_size].shape)

# The first row of groudtruth_data_rela_pos is already relative displacement to last position
y[0,:] = groudtruth_data_rela_pos.iloc[0,:]
for i in range(1, y.shape[0] ):
    y[i,:3] = groudtruth_data_rela_pos.iloc[i,:3] - groudtruth_data_rela_pos.iloc[i-1,:3]
    y[i,3:] = groudtruth_data_rela_pos.iloc[i,3:]

# Handle extreme small number
y[y<1e-6] = 1e-6
    
input_dim = X.shape[1]
output_dim = y.shape[1] 

X = X.reshape(total_num_sequence, series_length, input_dim)
y = y.reshape(total_num_sequence, series_length, output_dim)


# print ('Training input data shape:',train_X.shape)
# print ('Testing output data shape:',train_Y.shape)

# Use relative position here
# X = X[:, :, :3]
# y = y[:, :, :3]
print ('All input data shape:', X.shape)
print ('All output data shape:', y.shape)

train_val_split_ratio = 0.8
train_num_sequence = round( total_num_sequence*train_val_split_ratio)

train_X = np.copy(X[:train_num_sequence,...])
train_Y = np.copy(y[:train_num_sequence,...])

# train_val_X = train_X.reshape(1,-1,input_dim)
# train_val_Y = train_Y.reshape(1,-1,output_dim)

test_X = np.copy(X[train_num_sequence:,...])
test_Y = np.copy(y[train_num_sequence:,...])

new_train_X = np.concatenate((train_X, train_Y), axis=2)
new_train_Y = train_Y[:,1:,:]

All input data shape: (90, 400, 6)
All output data shape: (90, 400, 7)


In [6]:
# for indexi,i in enumerate(y):
#     for indexj,j in enumerate(i):
#         if (abs(j)<0.000001).any():
#             print(j, indexi, indexj)
#             print ('------')

In [7]:
# print(y[0,29,:])
# test_X.shape

In [8]:
from random import randint
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.rnn import BasicLSTMCell

log_dir = '../tmp/logs/sensor_fusion_LSTM_rel_pos'
training_steps = 1000
drift_thredshold = 0.5
batch_size = 4

class RollingData(object):
    ''' X should be of shape (num_of_batches, sequence_length, input_dim)
        Shuffle the data!!! Small training batches!!!
    '''
    def __init__(self, X, Y,):
        self.pointer = 0
        self.X = X
        self.Y = Y
        self._num_of_samples = X.shape[0]
#         self.train_len = X.shape[1]/2
    
    @property
    def num_of_samples(self):
        return self._num_of_samples

    def next_batch(self):
        # Generator random number in the range self._num_of_samples
        index = np.random.choice(self._num_of_samples, 4, replace=False)
        batch_X = self.X[index,:,:]
        batch_Y = self.Y[index,:,:]        
#         try:
#             train_X = self.X[self.pointer, :self.train_len, :]
#             train_Y = self.Y[self.pointer, :self.train_len, :]
#             test_X  = self.X[self.pointer, self.train_len:, :]
#             test_Y  = self.Y[self.pointer, self.train_len:, :]
#             self.pointer += 1
#         except IndexError:
#             print ("Index out of range. Index range:{} Query index{}".format(self.num_of_batches, self.pointer))
#             sys.exit(1)
        return (batch_X, batch_Y)
    
#     def reset_pointer(self):
#         self.pointer = 0

class IMU_LSTM_Model(object):
    def __init__(self, lstm_cell=128, batch_size=1):

        self.lstm    = BasicLSTMCell(128)    
#         self.hidden_state = tf.zeros([batch_size, 64], dtype=tf.float32)
#         self.output_state = tf.zeros([batch_size, 64], dtype=tf.float32)
        self.state = (tf.zeros([batch_size, 128], dtype=tf.float32),tf.zeros([batch_size, 128], dtype=tf.float32))
        
#         self.batch_size = batch_size
        print('hello, good initialize')
#         self.warm_up_loss = 0
#         self.predict_loss = 0
#         self.loss = self.warm_up_loss + 9*self.predict_loss
        
        
    def forward(self, X, mode=0):
#         with tf.variable_scope('fc1_before', reuse=True):
#             # Activation function here is tricky
# *** Change X 
        if mode ==0:
            self.X = X
        if mode ==1:
            self.X = tf.concat([X[:,:6],self.output], axis=1 )
        output1_before  = fully_connected(inputs=X, num_outputs=64, activation_fn= None)
#         with tf.variable_scope('fc2_before', reuse=True):
        output2_before  = fully_connected(inputs=output1_before, num_outputs=128, activation_fn= None)
#         with tf.variable_scope('lstm64',reuse=True):
        output_lstm, self.state = self.lstm(output2_before, self.state)
#         with tf.variable_scope('fc2_after',reuse=True):
        output_fc2 = fully_connected(inputs=output_lstm, num_outputs=128, activation_fn= None)
#         with tf.variable_scope('fc3_after',reuse=True):
        output_fc3 = fully_connected(inputs=output_fc2, num_outputs=64, activation_fn= None)
#         with tf.variable_scope('fc4_after', reuse=True):
        self.output = fully_connected(inputs=output_fc3, num_outputs=7, activation_fn= None)    #    
        return self.output
                
    # This method should be used in warm-up phase and between batches.
    def reset_state(self):
#         assert mode in (0,1), "mode should be either 0 or 1: {}".format(mode) 
        # Use mode 0 @ batch swtiching

        self.state = (tf.zeros([batch_size, 128], dtype=tf.float32),tf.zeros([batch_size, 128], dtype=tf.float32))
#             self.hidden_state = tf.zeros([batch_size, 64])
#             self.output_state = tf.zeros([batch_size, 64])
#             print(self.output_state)
        return self.state



def customized_loss(prediction, groundtruth, orientation_weight=0.10 ):
    # Orientation_weight should be decimal between 0 and 1, default=0.01
    pos_loss = tf.losses.mean_squared_error(prediction[:,:,:3],groundtruth[:,:,:3])
    # Step 1, normalize angle 
    normalized_orientation = tf.divide(prediction, tf.norm(prediction[:,:,3:], axis=2, keep_dims=True))
    ori_loss = tf.losses.mean_squared_error(normalized_orientation[:,:,3:], groundtruth[:,:,3:])
    # Step 2, get loss
    loss = 100*(pos_loss + orientation_weight*ori_loss)
    return loss, pos_loss, ori_loss



In [ ]:
tf.reset_default_graph()
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, shape=(batch_size, series_length, 13), name='x-input')
    Y = tf.placeholder(tf.float32, shape=(batch_size, series_length-1, 7), name='y-input')
#     mode = tf.placeholder(tf.int32, name='mode')
    
imu_lstm_model = IMU_LSTM_Model(batch_size=batch_size)
print(imu_lstm_model.lstm.state_size)


train_warm_up_pred = []
train_test_pred = []

# Because the last input, we don't have groudtruth, so we can not say its pred is right or wrong
for i in range(series_length-1):

    if i<series_length//2:
#         imu_lstm_model.set_state(mode=1, train_Y=Y[:,i,:])
#         train_test_pred.append(imu_lstm_model.forward(X[:,i,:]))  
#         _ = imu_lstm_model.set_state(mode=1, gt_train_Y = Y[:,series_length,:])
        train_warm_up_pred.append(imu_lstm_model.forward(X[:,i,:],  mode=0)) 
    else:
#         train_test_pred[:,i,:] = imu_lstm_model.forward(X[:,i,:])
        train_test_pred.append(imu_lstm_model.forward(X[:,i,:], mode=1)) 
#     if train_num_sequence == series_length-2:
#         zero_state = imu_lstm_model.reset_state()


print (len(train_warm_up_pred))
print (train_warm_up_pred[0].get_shape())

train_warm_up_pred = tf.reshape(train_warm_up_pred,[series_length//2,batch_size,output_dim])
train_warm_up_pred = tf.transpose(train_warm_up_pred, perm=[1,0,2])
train_test_pred = tf.reshape(train_test_pred,[series_length//2-1,batch_size,output_dim])
train_test_pred = tf.transpose(train_test_pred, perm=[1,0,2])

print (train_warm_up_pred.get_shape())


loss_warm_up, pos_loss_train, ori_loss_train = customized_loss(train_warm_up_pred,Y[:,:series_length//2,:])
loss_prediction, pos_loss, ori_loss = customized_loss(train_test_pred, Y[:,series_length//2:,:])
total_loss = 0.2*loss_warm_up + 0.8*loss_prediction

train_step = tf.train.AdamOptimizer().minimize(loss_prediction)

hello, good initialize
LSTMStateTuple(c=128, h=128)
200
(4, 7)
(4, 200, 7)


In [ ]:
#-------***************--------------#

tf.summary.scalar('total_loss',total_loss)
tf.summary.scalar('loss_warm_up', loss_warm_up)
tf.summary.scalar('pos_loss_train', pos_loss_train)
tf.summary.scalar('ori_loss_train', ori_loss_train)
tf.summary.scalar('pred_loss',loss_prediction)
tf.summary.scalar('pos_loss',pos_loss)
tf.summary.scalar('ori_loss', ori_loss)
merged = tf.summary.merge_all()

saver = tf.train.Saver()


sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)
train_rolling = RollingData(new_train_X, new_train_Y)
train_writer = tf.summary.FileWriter(log_dir+'/train', sess.graph, flush_secs=2, max_queue=2)


for e in range(3000):
    print ('Epoch {}/{} '.format(e, training_steps), end='\r')
    train_X_batch, train_Y_batch = train_rolling.next_batch()
    
    _,pred_loss, _= sess.run([train_step,loss_prediction, total_loss], feed_dict={X:train_X_batch, Y:train_Y_batch})
    zero_state = sess.run(imu_lstm_model.reset_state())
    print ("Prediction loss at step {} is: {}".format(e, pred_loss))
#     print ('Output state at the end of step {} is {}'.format(e, zero_state))

    summary,  = sess.run([merged], feed_dict={X:train_X_batch, Y:train_Y_batch})
    train_writer.add_summary(summary,e) 
    if e%300 ==0:
        save_path = saver.save(sess,'../tmp/logs/sensor_fusion_LSTM_rel_pos/model_%d.ckpt'%(e))
#     print('Hello test positioni correct ratio:{} \t and loss:{}'.format(test_correct_ratio, test_loss))
    
sess.close()

Prediction loss at step 0 is: 11.657685279846191
Prediction loss at step 1 is: 7.765103816986084
Prediction loss at step 2 is: 8.129000663757324
Prediction loss at step 3 is: 6.060035705566406
Prediction loss at step 4 is: 4.514664649963379
Prediction loss at step 5 is: 3.8734469413757324
Prediction loss at step 6 is: 3.7855207920074463
Prediction loss at step 7 is: 3.6128692626953125
Prediction loss at step 8 is: 3.4527950286865234
Prediction loss at step 9 is: 3.5951004028320312
Prediction loss at step 10 is: 3.4765264987945557
Prediction loss at step 11 is: 2.778843402862549
Prediction loss at step 12 is: 2.7393362522125244
Prediction loss at step 13 is: 2.735983371734619
Prediction loss at step 14 is: 2.678969383239746
Prediction loss at step 15 is: 2.7016830444335938
Prediction loss at step 16 is: 2.7466416358947754
Prediction loss at step 17 is: 2.5328562259674072
Prediction loss at step 18 is: 2.2545382976531982
Prediction loss at step 19 is: 2.152743101119995
Prediction loss at

In [ ]:

# # Drift + means prediction is always more extreme;  drift - prediction is mild (in relative percentage)
# with tf.name_scope('metrics'):
#     drift = (preds[:,:,:3]-Y[:,:,:3])/Y[:,:,:3]
#     drift_truth_table = tf.to_float(tf.abs(drift) < drift_thredshold)
#     drift_within_t = tf.reduce_prod(drift_truth_table, axis=2)
#     correct_pred_ratio = tf.reduce_mean(drift_within_t)
    
# tf.summary.histogram('drift', drift)
# tf.summary.histogram('drift_within_threshold', drift_within_t)   
# tf.summary.scalar('loss',loss)
# tf.summary.scalar('correct_pred_ratio', correct_pred_ratio)
# merged = tf.summary.merge_all()

# # my_metrics = my_metrics(Y, preds)
# # p_x = my_metrics[0]
